In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from google.colab.patches import cv2_imshow
import torch.nn.functional as F





In [ ]:
if torch.cuda.is_available():
  device='cuda:0'
else:
  device='cpu'

In [ ]:
# Definir transformaciones para los datos de entrenamiento y prueba
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Descargar y cargar los datos de entrenamiento y prueba
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)





In [ ]:
# Definir la red neuronal
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)  # 1 input channel (grayscale), 6 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)  # 6 input channels, 16 output channels, 5x5 square convolution kernel
        self.fc1 = nn.Linear(16*5*5, 120)  # 16*5*5 input features, 120 output features
        self.fc2 = nn.Linear(120, 84)  # 120 input features, 84 output features
        self.fc3 = nn.Linear(84, 10)  # 84 input features, 10 output features (for the 10 classes of MNIST)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply the first convolutional layer followed by ReLU activation
        x = F.max_pool2d(x, 2)  # Apply 2x2 max pooling
        x = F.relu(self.conv2(x))  # Apply the second convolutional layer followed by ReLU activation
        x = F.max_pool2d(x, 2)  # Apply 2x2 max pooling
        x = x.view(-1, 16*5*5)  # Flatten the tensor into a vector
        x = F.relu(self.fc1(x))  # Apply the first fully connected layer followed by ReLU activation
        x = F.relu(self.fc2(x))  # Apply the second fully connected layer followed by ReLU activation
        x = self.fc3(x)  # Apply the final fully connected layer
        return x



In [ ]:
# Instanciar el modelo, el criterio de pérdida y el optimizador
net = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# Entrenamiento del modelo
epochs = 5
net=net.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()           # Limpiar los gradientes
        outputs = net(inputs)           # Forward pass
        loss = criterion(outputs, labels) # Calcular la pérdida
        loss.backward()                 # Backward pass
        optimizer.step()                # Optimización
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

# Evaluación del modelo
correct = 0
total = 0
net=net.eval()
for inputs, labels in testloader:
    inputs=inputs.to(device)
    labels=labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

In [ ]:
import time
import numpy as np
indiv_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)
i=0
max_ejemp=10
for image,label in indiv_loader:
  if i<10:
    res=net(image.to(device))
    _,pred=torch.max(res,1)
    print(50*'-')
    print(f'Predicción: {pred.item()}')
    cv2_imshow((255*image[0][0].numpy()).astype(np.uint8))
    time.sleep(2)
    i+=1
  else:
    print('Ha finalizado el demo')
    break